In [ ]:
# This is the fully loaded code with the netcdf files and creation of heat stree\WBGT with the Global World bank datasets.
# It will provide the 

#!pip install netCDF4 h5py h5netcdf xarray rasterio pandas numpy

import os
import numpy as np
import pandas as pd
import xarray as xr
from rasterio.transform import from_origin
import rasterio

# ------------------ USER: set these file paths ------------------
# Historical (baseline 2014)
tas_hist = r"C:\Users\DELL\Desktop\Cognitud\Heat Stress Project\timeseries-tasmax-annual-mean_cmip6-x0.25_ensemble-all-historical_timeseries-smooth_median_1950-2014.nc"
hurs_hist = r"C:\Users\DELL\Desktop\Cognitud\Heat Stress Project\timeseries-hurs-annual-mean_cmip6-x0.25_ensemble-all-historical_timeseries-smooth_median_1950-2014.nc"

# Scenarios (2015-2100)
tas_ssp126 = r"C:\Users\DELL\Desktop\Cognitud\Heat Stress Project\timeseries-tasmax-annual-mean_cmip6-x0.25_ensemble-all-ssp126_timeseries-smooth_median_2015-2100.nc"
tas_ssp245 = r"C:\Users\DELL\Desktop\Cognitud\Heat Stress Project\timeseries-tasmax-annual-mean_cmip6-x0.25_ensemble-all-ssp245_timeseries-smooth_median_2015-2100.nc"
tas_ssp585 = r"C:\Users\DELL\Desktop\Cognitud\Heat Stress Project\timeseries-tasmax-annual-mean_cmip6-x0.25_ensemble-all-ssp585_timeseries-smooth_median_2015-2100.nc"

hurs_ssp126 = r"C:\Users\DELL\Desktop\Cognitud\Heat Stress Project\timeseries-hurs-annual-mean_cmip6-x0.25_ensemble-all-ssp126_timeseries-smooth_median_2015-2100.nc"
hurs_ssp245 = r"C:\Users\DELL\Desktop\Cognitud\Heat Stress Project\timeseries-hurs-annual-mean_cmip6-x0.25_ensemble-all-ssp245_timeseries-smooth_median_2015-2100.nc"
hurs_ssp585 = r"C:\Users\DELL\Desktop\Cognitud\Heat Stress Project\timeseries-hurs-annual-mean_cmip6-x0.25_ensemble-all-ssp585_timeseries-smooth_median_2015-2100.nc"
# ---------------------------------------------------------------

OUT_DIR_BASE = r"C:\Users\DELL\Desktop\Cognitud\Heat Stress Project\Specific_Maps"
os.makedirs(OUT_DIR_BASE, exist_ok=True)

SCENARIOS = {
    "ssp126": {"tas": tas_ssp126, "hurs": hurs_ssp126},
    "ssp245": {"tas": tas_ssp245, "hurs": hurs_ssp245},
    "ssp585": {"tas": tas_ssp585, "hurs": hurs_ssp585},
}
YEARS = [2030, 2050, 2100]

# ---------------- Helpers & thresholds ----------------
def open_ds_safe(ncfile):
    if not os.path.exists(ncfile):
        raise FileNotFoundError(f"NetCDF not found: {ncfile}")
    for eng in ("netcdf4", "h5netcdf"):
        try:
            return xr.open_dataset(ncfile, engine=eng, decode_cf=True)
        except Exception:
            pass
    return xr.open_dataset(ncfile, decode_cf=True)

def normalize_latlon(ds):
    rename = {}
    if "latitude" in ds.coords:  rename["latitude"]  = "lat"
    if "Latitude" in ds.coords:  rename["Latitude"]  = "lat"
    if "longitude" in ds.coords: rename["longitude"] = "lon"
    if "Longitude" in ds.coords: rename["Longitude"] = "lon"
    if rename:
        ds = ds.rename(rename)
    return ds

def select_nearest_year(da, target_year):
    time_name = None
    for cand in ("time","year","Year"):
        if cand in da.coords:
            time_name = cand
            break
    if time_name is None:
        return da
    coord = da[time_name]
    if np.issubdtype(coord.dtype, np.datetime64):
        years = np.array([int(str(t.astype('datetime64[Y]'))[:4]) for t in coord.values])
    else:
        years = np.array([int(v) for v in coord.values])
    idx = int(np.argmin(np.abs(years - target_year)))
    return da.isel({time_name: idx})

def write_tif(path, array, lons, lats, nodata, dtype, crs="EPSG:4326"):
    lats = np.asarray(lats); lons = np.asarray(lons)
    if len(lons) < 2 or len(lats) < 2:
        raise ValueError("Need at least 2 unique lons and lats to set resolution")
    res_lon = float(lons[1] - lons[0]); res_lat = float(lats[0] - lats[1])
    transform = from_origin(float(lons.min()), float(lats.max()), res_lon, res_lat)
    profile = {
        "driver": "GTiff",
        "height": array.shape[0],
        "width": array.shape[1],
        "count": 1,
        "crs": crs,
        "transform": transform,
        "dtype": dtype,
        "nodata": nodata
    }
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with rasterio.open(path, "w", **profile) as dst:
        dst.write(array, 1)

# Category A (Comfortable .. Danger) - used for scenario zones
def wbgt_category_A(wbgt):
    if wbgt < 22:
        return "Very low"
    elif wbgt <= 24:
        return "Low"
    elif wbgt <= 30:
        return "Moderate"
    elif wbgt <= 33:
        return "High"
    else:
        return "Very high"

def wbgt_zones_A(wb):
    codes = np.zeros_like(wb, dtype=np.int16)
    codes[wb < 22] = 1
    codes[(wb >= 22) & (wb <= 24)] = 2
    codes[(wb > 24) & (wb <= 30)] = 3
    codes[(wb > 30) & (wb <= 33)] = 4
    codes[wb > 33] = 5
    return codes

# Baseline intensity (Very low .. Very high) - used for baseline zones
def wbgt_intensity_2014_label(wbgt):
    if wbgt < 22:
        return "Very low"
    elif wbgt <= 24:
        return "Low"
    elif wbgt <= 30:
        return "Moderate"
    elif wbgt <= 33:
        return "High"
    else:
        return "Very high"

def wbgt_intensity_2014_code(wb):
    codes = np.zeros_like(wb, dtype=np.int16)
    codes[wb < 22] = 1
    codes[(wb >= 22) & (wb <= 24)] = 2
    codes[(wb > 24) & (wb <= 30)] = 3
    codes[(wb > 30) & (wb <= 33)] = 4
    codes[wb > 33] = 5
    return codes

# ---------------- Process baseline 2014 (global) ----------------
print("Processing baseline (2014)...")
ds_tas_hist = open_ds_safe(tas_hist); ds_tas_hist = normalize_latlon(ds_tas_hist)
ds_hurs_hist = open_ds_safe(hurs_hist); ds_hurs_hist = normalize_latlon(ds_hurs_hist)

tas_var = list(ds_tas_hist.data_vars.keys())[0]
hurs_var = list(ds_hurs_hist.data_vars.keys())[0]
da_tas_hist = ds_tas_hist[tas_var]
da_hurs_hist = ds_hurs_hist[hurs_var]

tas_2014 = select_nearest_year(da_tas_hist, 2014)
hurs_2014 = select_nearest_year(da_hurs_hist, 2014)

tas_df = tas_2014.to_dataframe().reset_index()
hurs_df = hurs_2014.to_dataframe().reset_index()

tas_col = tas_2014.name if tas_2014.name else "tas"
hurs_col = hurs_2014.name if hurs_2014.name else "hurs"
tas_df = tas_df.rename(columns={tas_2014.name: tas_col})
hurs_df = hurs_df.rename(columns={hurs_2014.name: hurs_col})

global_df = pd.merge(tas_df, hurs_df, on=["lat","lon"], how="inner")

if global_df[hurs_col].max() <= 1.0 + 1e-6:
    global_df[hurs_col] = global_df[hurs_col] * 100.0
if global_df[tas_col].mean() > 150:
    global_df[tas_col] = global_df[tas_col] - 273.15

# WBGT
T = global_df[tas_col].astype(float)
RH = global_df[hurs_col].astype(float)
e = (RH / 100.0) * 6.105 * np.exp((17.27 * T) / (237.7 + T))
global_df["WBGT_Liljegren"] = 0.567 * T + 0.393 * e + 3.94

# Baseline intensity zones (codes 1..5) + labels
global_df["WBGT_Intensity_Label"] = global_df["WBGT_Liljegren"].apply(wbgt_intensity_2014_label)
global_df["WBGT_Zones_1to5"] = wbgt_intensity_2014_code(global_df["WBGT_Liljegren"].values)

# Save baseline CSV
base_dir = os.path.join(OUT_DIR_BASE, "baseline_2014")
os.makedirs(base_dir, exist_ok=True)
out_csv_base = os.path.join(base_dir, "baseline_wbgt_2014.csv")
global_df.to_csv(out_csv_base, index=False)

# Rasterize baseline WBGT + zones (int codes)
lats = np.sort(global_df["lat"].unique())[::-1]
lons = np.sort(global_df["lon"].unique())

wbgt_grid = global_df.pivot(index="lat", columns="lon", values="WBGT_Liljegren").reindex(index=lats, columns=lons).values
zones_grid = global_df.pivot(index="lat", columns="lon", values="WBGT_Zones_1to5").reindex(index=lats, columns=lons).values

wbgt_grid = np.asarray(wbgt_grid, dtype=np.float32)
zones_grid = np.asarray(zones_grid, dtype=np.float32)

out_wbgt_tif = os.path.join(base_dir, "wbgt_baseline_2014.tif")
out_zones_tif = os.path.join(base_dir, "wbgt_zones_baseline_2014.tif")
write_tif(out_wbgt_tif, np.where(np.isnan(wbgt_grid), np.nan, wbgt_grid), lons, lats, nodata=np.nan, dtype="float32")
write_tif(out_zones_tif, np.where(np.isnan(zones_grid), 0, zones_grid).astype(np.int16), lons, lats, nodata=0, dtype="int16")

print("Saved baseline maps:", out_wbgt_tif, out_zones_tif)

# ---------------- Process each scenario-year (global) ----------------
for scen, files in SCENARIOS.items():
    print(f"\nProcessing scenario: {scen}")
    try:
        ds_tas = open_ds_safe(files["tas"]); ds_tas = normalize_latlon(ds_tas)
        ds_hurs = open_ds_safe(files["hurs"]); ds_hurs = normalize_latlon(ds_hurs)
    except Exception as e:
        print(f" Failed to open files for {scen}: {e}")
        continue

    tas_var = list(ds_tas.data_vars.keys())[0]
    hurs_var = list(ds_hurs.data_vars.keys())[0]
    da_tas = ds_tas[tas_var]
    da_hurs = ds_hurs[hurs_var]

    for year in YEARS:
        print(f" Year: {year}")
        try:
            tas_yr = select_nearest_year(da_tas, year)
            hurs_yr = select_nearest_year(da_hurs, year)
        except Exception as e:
            print(f"  Year selection error: {e}")
            continue

        tas_df = tas_yr.to_dataframe().reset_index()
        hurs_df = hurs_yr.to_dataframe().reset_index()
        tas_col = tas_yr.name if tas_yr.name else "tas"
        hurs_col = hurs_yr.name if hurs_yr.name else "hurs"
        tas_df = tas_df.rename(columns={tas_yr.name: tas_col})
        hurs_df = hurs_df.rename(columns={hurs_yr.name: hurs_col})

        global_df = pd.merge(tas_df, hurs_df, on=["lat","lon"], how="inner")

        if global_df[hurs_col].max() <= 1.0 + 1e-6:
            global_df[hurs_col] = global_df[hurs_col] * 100.0
        if global_df[tas_col].mean() > 150:
            global_df[tas_col] = global_df[tas_col] - 273.15

        # WBGT
        T = global_df[tas_col].astype(float)
        RH = global_df[hurs_col].astype(float)
        e = (RH / 100.0) * 6.105 * np.exp((17.27 * T) / (237.7 + T))
        global_df["WBGT_Liljegren"] = 0.567 * T + 0.393 * e + 3.94

        # Category A zones for scenarios (Comfortable -> Danger)
        global_df["Heat_Stress_Category"] = global_df["WBGT_Liljegren"].apply(wbgt_category_A)
        global_df["WBGT_Zones_1to5"] = wbgt_zones_A(global_df["WBGT_Liljegren"].values)

        # Save CSV & TIFFs in scenario/year folder
        outdir = os.path.join(OUT_DIR_BASE, scen, str(year))
        os.makedirs(outdir, exist_ok=True)
        out_csv = os.path.join(outdir, f"{scen}_{year}_wbgt.csv")
        global_df.to_csv(out_csv, index=False)

        lats = np.sort(global_df["lat"].unique())[::-1]
        lons = np.sort(global_df["lon"].unique())
        if len(lats) < 2 or len(lons) < 2:
            print("  Not enough grid points to rasterize; CSV saved only.")
            continue

        wbgt_grid = global_df.pivot(index="lat", columns="lon", values="WBGT_Liljegren").reindex(index=lats, columns=lons).values
        zones_grid = global_df.pivot(index="lat", columns="lon", values="WBGT_Zones_1to5").reindex(index=lats, columns=lons).values

        wbgt_grid = np.asarray(wbgt_grid, dtype=np.float32)
        zones_grid = np.asarray(zones_grid, dtype=np.float32)

        out_wbgt = os.path.join(outdir, f"{scen}_{year}_wbgt.tif")
        out_zones = os.path.join(outdir, f"{scen}_{year}_zones.tif")
        write_tif(out_wbgt, np.where(np.isnan(wbgt_grid), np.nan, wbgt_grid), lons, lats, nodata=np.nan, dtype="float32")
        write_tif(out_zones, np.where(np.isnan(zones_grid), 0, zones_grid).astype(np.int16), lons, lats, nodata=0, dtype="int16")

        print("  Saved:", out_wbgt, out_zones, "and CSV:", out_csv)

print("\nDone. All specific TIFFs/CSVs are under:", OUT_DIR_BASE)


In [ ]:
"""
batch_clip_recursive.py

Recursively clip TIFFs under your Specific_Maps folder by the provided shapefile.
Only writes clipped TIFFs (no PNGs).
"""
## I will clip it only the land area of the stress using the Shape file.

import os
from pathlib import Path
import warnings

import rasterio
from rasterio.mask import mask
from rasterio.enums import ColorInterp
import geopandas as gpd
from shapely.geometry import mapping, box, shape

# ===================== USER PATHS (EDIT if needed) =====================
MAPS_ROOT = r"C:\Users\DELL\Desktop\Cognitud\Heat Stress Project\Specific_Maps"
SHAPEFILE = r"C:\Users\DELL\Desktop\Cognitud\world_shp\geoBoundariesCGAZ_ADM0\geoBoundariesCGAZ_ADM0.shp"
OUT_ROOT  = r"C:\Users\DELL\Desktop\Cognitud\Heat Stress Project\Specific_Maps\Clipped_Output"
# =====================================================================

os.makedirs(OUT_ROOT, exist_ok=True)

# Parameters
CROP_TO_SHAPE = True   # True -> crop to polygon bbox; False -> keep original extent & mask
USE_UNION_GEOM = True  # dissolve all polygons into one

# Load shapefile and dissolve to one polygon (union)
gdf = gpd.read_file(SHAPEFILE)
if gdf.empty:
    raise RuntimeError("Shapefile empty or missing. Check path and .shx/.dbf files.")
union_geom = gdf.unary_union if USE_UNION_GEOM else gdf.geometry

# find tifs recursively
tif_paths = sorted(Path(MAPS_ROOT).rglob("*.tif"))
if not tif_paths:
    raise SystemExit(f"No .tif files found under: {MAPS_ROOT}")

print(f"Found {len(tif_paths)} TIFF(s). Processing...")

for tif in tif_paths:
    rel = tif.relative_to(MAPS_ROOT)
    out_tif = Path(OUT_ROOT) / rel.parent / f"clipped_{tif.name}"
    out_tif.parent.mkdir(parents=True, exist_ok=True)

    print("\n----")
    print("Input:", tif)
    try:
        with rasterio.open(tif) as src:
            src_nodata = src.nodatavals[0] if src.nodatavals else None

            # Reproject vector to raster CRS if needed
            if gdf.crs != src.crs:
                geom_reproj = gpd.GeoSeries([union_geom], crs=gdf.crs).to_crs(src.crs).iloc[0]
                geom_for_mask = [mapping(geom_reproj)]
            else:
                geom_for_mask = [mapping(union_geom)]

            # Quick intersection check
            raster_box = box(*src.bounds)
            shp_shape = shape(geom_for_mask[0])
            if not raster_box.intersects(shp_shape):
                warnings.warn(" -> No intersection. Skipping this tile.")
                continue

            # Perform mask
            clipped, out_transform = mask(
                src, geom_for_mask,
                crop=CROP_TO_SHAPE, filled=True, nodata=src_nodata
            )

            # Count valid pixels
            valid_pixels = 0
            for b in range(clipped.shape[0]):
                band = clipped[b]
                if src_nodata is not None:
                    valid = (band != src_nodata).sum()
                else:
                    valid = (~(band != band)).sum()  # count non-NaN
                valid_pixels = max(valid_pixels, valid)

            if valid_pixels == 0:
                warnings.warn("Clipped result contains 0 valid pixels. Skipping write.")
                continue

            # Write output and attempt to preserve colormap for 1-band paletted rasters
            profile = src.profile.copy()
            profile.update({
                "height": clipped.shape[1],
                "width": clipped.shape[2],
                "transform": out_transform,
                "count": clipped.shape[0],
            })
            if src_nodata is not None:
                profile["nodata"] = src_nodata

            with rasterio.open(out_tif, "w", **profile) as dst:
                dst.write(clipped)
                try:
                    cm = src.colormap(1)
                    if cm:
                        dst.write_colormap(1, cm)
                        dst.colorinterp = (ColorInterp.palette,)
                        print(" Preserved color map (palette).")
                except Exception:
                    pass

            print(" ✅ Wrote:", out_tif)

    except Exception as e:
        warnings.warn(f"Failed processing {tif}: {e}")

print("\nAll done. Outputs in:", OUT_ROOT)


In [ ]:
## Smoothening the Pixels for better look
import os
import rasterio
from rasterio.features import shapes, rasterize
from shapely.geometry import shape
import geopandas as gpd
from rasterio.transform import Affine
import numpy as np

# ========== CONFIG ==========
IN_DIR  = r"C:\Users\DELL\Desktop\Cognitud\Heat Stress Project\OK Maps"
OUT_DIR = r"C:\Users\DELL\Desktop\Cognitud\Heat Stress Project\OK Maps_smooth"
# If you prefer SMOOTH in pixels, set USE_PIXELS=True and specify SMOOTH_PIXELS.
USE_PIXELS = True
SMOOTH_PIXELS = 2         # how many pixels to smooth (try 1,2,3,5)
# If USE_PIXELS=False then SMOOTH is interpreted in CRS units (degrees or meters)
SMOOTH = 1.5
# =================================

os.makedirs(OUT_DIR, exist_ok=True)

def pixel_size(transform: Affine):
    """Return approximate pixel size (max of xres and abs(yres)) in CRS units"""
    xres = abs(transform.a)
    yres = abs(transform.e)
    return max(xres, yres)

def smooth_edges(data, transform, nodata=None, smooth_pixels=None, smooth_crs=None):
    # Mask = all valid pixels (not nodata)
    if nodata is not None:
        valid_mask = (data != nodata)
    else:
        # treat NaN as nodata for float rasters, otherwise consider >0 as valid
        if np.issubdtype(data.dtype, np.floating):
            valid_mask = ~np.isnan(data)
        else:
            valid_mask = (data != 0)

    # Vectorize zones
    results = (
        {"properties": {"val": v}, "geometry": s}
        for s, v in shapes(data, mask=valid_mask, transform=transform)
    )
    gdf = gpd.GeoDataFrame.from_features(results)

    if gdf.empty:
        # nothing to do
        return data.copy()

    # World outline = union of all polygons (cleaned)
    world_outline = gdf.unary_union.buffer(0)

    # determine buffer distance to use
    if smooth_pixels is not None:
        # convert pixel count -> CRS units
        px = pixel_size(transform)
        buf = smooth_pixels * px
    else:
        buf = smooth_crs if smooth_crs is not None else SMOOTH

    # Smooth polygons using buffer in/out
    # use buffer(,) on GeoDataFrame (applies per row) — safer if many classes
    try:
        gdf["geometry"] = gdf.geometry.buffer(buf).buffer(-buf)
    except Exception:
        # fallback: operate on unary_union per geometry (slower)
        gdf["geometry"] = [geom.buffer(buf).buffer(-buf) if geom is not None else None for geom in gdf.geometry]

    # Clip smoothed polygons back to world outline
    gdf["geometry"] = gdf["geometry"].intersection(world_outline)

    # Rasterize back. Use nodata as fill if provided, else 0.
    fillval = nodata if nodata is not None else 0
    # Build list of (geom, val) skipping empty geometries
    items = [(geom, val) for geom, val in zip(gdf.geometry, gdf["val"]) if geom is not None and not geom.is_empty]

    if not items:
        return data.copy()

    out_data = rasterize(
        items,
        out_shape=data.shape,
        transform=transform,
        fill=fillval,
        dtype=data.dtype,
    )
    return out_data

# ========== LOOP OVER ALL TIFFS ==========
for fname in sorted(os.listdir(IN_DIR)):
    if not fname.lower().endswith(".tif"):
        continue

    in_path  = os.path.join(IN_DIR, fname)
    out_path = os.path.join(OUT_DIR, fname.replace(".tif", "_smooth.tif"))

    with rasterio.open(in_path) as src:
        data = src.read(1)
        profile = src.profile.copy()
        transform = src.transform
        nodata = profile.get("nodata", None)

        if USE_PIXELS:
            smoothed = smooth_edges(data, transform, nodata=nodata, smooth_pixels=SMOOTH_PIXELS)
        else:
            smoothed = smooth_edges(data, transform, nodata=nodata, smooth_crs=SMOOTH)

        # write out (preserve dtype and nodata)
        profile.update(dtype=smoothed.dtype, count=1)
        if nodata is not None:
            profile["nodata"] = nodata

    with rasterio.open(out_path, "w", **profile) as dst:
        dst.write(smoothed, 1)

    print(f"✅ Smoothed saved: {out_path}")

print("🎯 All TIFFs processed — smooth edges, world boundary preserved.")
